## Polygon Filtering

In [66]:
import numpy as np
import pandas as pd
from datetime import timedelta

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

An efficient way to test if a point is contained within a polygon is [described on this page](http://alienryderflex.com/polygon/). There are even implementations on that page (though not in Python).

### Pipeline for Data Preprocessing

In [79]:
####### helper functions

def in_square(r_lat, r_lon, sq):
    return ((r_lat >=  sq[0]) & (r_lat <= sq[1]) & (r_lon >= sq[2]) & (r_lon <= sq[3]))

#### function for marking the square
def mark_square(r, dict_label_square):
    r_lat = r['latitude']
    r_lon = r['longitude']
    for label, square in dict_label_square.items():
        if in_square(r_lat, r_lon, square):
            return label
    
    return "not found"

def parse_grid_file(grid_file: str):
    df = pd.read_csv(grid_file)
    vals = df.values
    str_to_arr = lambda s: np.array([float(x) for x in s.split(',')])
    vals = np.array([[str_to_arr(x2) for x2 in x1] for x1 in vals])
    return vals

def grid_to_dict(grid_csv):
    grid_arr = parse_grid_file(grid_csv)
    reshaped_grid = grid_arr.reshape(-1,8)
    grid_dict = {}
    for i in range(len(reshaped_grid)):
        lat_lon = np.sort(list(set(reshaped_grid[i])))
        grid_dict[i] = lat_lon
    
    return grid_dict


In [134]:
##### main pipeline

def viirs_pipe(viirs_csv, grid_csv):
    viirs = pd.read_csv(viirs_csv)
    viirs = viirs[['latitude', 'longitude', 'acq_date', 'acq_time']]
    grid_dict = grid_to_dict(grid_csv)
    viirs['region'] = viirs.apply(lambda r: mark_square(r, grid_dict), axis=1)
#     viirs['match_date'] = pd.to_datetime(small_clean_viirs_2019['acq_time']) - timedelta(n_days)
    return viirs

def img_pipe(img_csv, n_weeks = 5, n_day_range = 7):
    img = pd.read_csv(img_csv)
    img['date'] = pd.to_datetime(img['date'])   
    img['fire_start'] = img['date'] + timedelta(n_weeks*7)
    img['fire_end'] = img['date'] + timedelta(n_weeks*7+n_day_range)
    return img
    

memory_dict = {}
def custom_merge(row, vrs):
    
    unique_event = (row['fire_start'], row['fire_end'],row['rect_id'])

    if unique_event not in memory_dict:
        events = (vrs[(vrs['acq_date']>= row['fire_start']) & 
                      (vrs['acq_date']<= row['fire_end']) & 
                      (vrs['region'] == row['rect_id'])])
        num_fires = len(events)
        memory_dict[unique_event] = num_fires
        print(num_fires)
    else:
        num_fires = memory_dict[unique_event]
        
    
    return num_fires

## Data Preprocessing

#### Add range to the img data

In [139]:
img_1 =img_pipe("img/1_img.csv", n_weeks = 5, n_day_range = 7)

In [140]:
img_2 =img_pipe("img/2_img.csv", n_weeks = 5, n_day_range = 7)

In [141]:
img_3 =img_pipe("img/3_img.csv", n_weeks = 5, n_day_range = 7)

In [142]:
img_4 =img_pipe("img/4_img.csv", n_weeks = 5, n_day_range = 7)

In [147]:
min(img_2['date'])

Timestamp('2014-01-04 00:00:00')

#### Preprocessing VIIRS

In [148]:
viirs_2014 = viirs_pipe("data/viirs-2014.csv", 'data/grid.csv')

In [149]:
viirs_2015 = viirs_pipe("data/viirs-2015.csv", 'data/grid.csv')

In [150]:
viirs_2016 = viirs_pipe("data/viirs-2016.csv", 'data/grid.csv')

In [151]:
viirs_2017 = viirs_pipe("data/viirs-2017.csv", 'data/grid.csv')

In [152]:
viirs_2018 = viirs_pipe("data/viirs-2018.csv", 'data/grid.csv')

In [153]:
viirs_2019 = vrs

In [156]:
viirs = pd.concat([viirs_2014, viirs_2015, viirs_2016, viirs_2017, viirs_2018, viirs_2019], axis=0)

In [158]:
len(viirs)

2116032

In [159]:
viirs = viirs[viirs['region'] != "not found"]

In [160]:
len(viirs)

311185

In [163]:
viirs.to_csv('clean-viirs.csv')

In [166]:
viirs['acq_date'] = pd.to_datetime(viirs['acq_date'])

### Produce the labels

In [167]:
img_1['fire_counts'] = img_1.apply(lambda r: custom_merge(r, viirs), axis = 1)

0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
1
1
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
4
2
1
1
0
0
0
0
0
0
0
1
0
3
0
0
1
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
1
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
1
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
9
11
20
13
1
0
0
0
0
0
0
0
0
0
0
0
0
3
11
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
5
0
10
9
0
3
2
0
0
0
0
0
0
0
0
0
0
0
1
2
6
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
1
0
0
0
0
0
0
0
0
0
0
0
0
0
4
4
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
4
3
15
2
0
1
0
0
0
0
0
0
0
0
0
0
0
0
14
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
2
0
5
1
6
15
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
6
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
6
1
2
0
0
0
0
0
0
0
0
0
0
0
1
3
8
2
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0


0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
1
2
0
0
0
0
0
0
0
0
0
0
0
1
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
4
0
0
0
0
0
0


0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
3
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
7
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
47
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
4
2
0
0
0
0
0
0
0
0
0
0
0
1
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
10
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


2
7
4
0
0
0
0
0
0
0
0
0
0
1
0
1
0
1
0
2
1
0
0
0
0
0
0
0
0
0
0
0
1
10
4
0
1
0
0
0
0
0
0
0
0
4
0
0
0
3
6
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
1
4
0
1
3
0
4
0
0
0
0
0
0
0
0
0
0
0
2
0
2
1
2
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
2
2
1
3
2
0
0
1
0
0
0
0
0
0
0
1
0
3
2
1
0
0
2
0
1
0
0
0
0
0
0
0
0
0
2
2
8
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
2
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
7
0
0
2
0
0
0
0
0
0
0
0
0
0
0
5
0
0
0
0
0
0
0
0
0
0
0
0
0
7
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
2
0
0
0
0
0
0
0
0
0
0
0
0
5
10
0
0
0
0
0
0
0
0
0
0
0
0
4
0
2
6
3
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
8
4
11
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
2
0
5
12
4
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
3
6
0
0
0
0
0
0
0
0
0
0
0
2
1
2
0
0
0
0
0
0
0
0
0
0
0
0
0
4
0
0
4
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
2
0
0
0
0
0
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
1
6
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
2
6
0
0
0
0
0
0
0
0
0
0
0
0
1
4
2
0
0
0
0
0
0
0
0
0
0
4
0
1
0
3
10
0
0
0

0
0
0
0
0
0
0
0
0
0
0
0
0
0
10
1
0
2
0
0
0
0
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
1
0
0
6
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
9
0
0
0
0
0
0
0
0
3
0
0
0
0
7
3
0
0
0
0
0
0
0
0
0
1
0
0
2
2
2
5
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
1
1
3
5
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
2
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
11
13
1
0
0
1
0
0
0
0
0
0
0
0
0
0
2
4
0
7
7
0
0
0
0
0
0
0
0
0
0
0
0
0
4
0
0
0
0
0
0
0
0
0
0
0
0
3
0
7
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
2
0
0
0
1
1
0
0
0
0
1
0
0
0
0
2
0
0
5
0
0
0
0
0
0
0
0
0
0
0
0
4
6
1
1
0
0
0
1
0
0
0
0
0
0
0
2
0
2
0
7
0
0
0
1
0
0
0
0
0
0
0
0
0
4
2
0
2
0
0
0
0
0
0
0
2
2
0
2
2
0
4
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
1
0
0
1
2
3
0
0
0
0
0
0
0
0
0
0
4
1
0
3
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
4
2
6
9
0
3
4
0
0
0
0
1
0
0
0
0
0
0
2
4
2
3
5
0
0
0
0
0
0
0
1
0
0
1
0
0
0
0
0
2
0
1
0
0
0
0
0
1
0
2
0
4
1
1
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
2
0
1
2
0
3
0
0
0
0
0
0
0
0
0
1
2
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
5
1
0
0
0
0
0
1
0
0
0
0

...

0
0
0
1
0
6
0
0
0
0
0
0
0
0
0
0
2
0
1
0
2
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
2
0
0
0
0
0
0
0
0
0
0
1
6
0
11
0
0
0
0
0
0
0
0
0
0
5
1
0
3
1
7
0
4
0
0
0
0
0
0
0
0
0
1
0
0
1
0
4
0
0
0
0
0
0
0
0
0
2
2
1
3
14
0
0
0
0
0
0
0
0
0
0
1
0
0
2
2
4
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
5
0
0
0
2
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
1
0
0
0
2
3
2
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
12
0
0
0
0
1
0
0
0
0
0
0
1
0
6
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
4
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
2
5
0
1
0
4
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
1
1
1
0
2
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
4
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
8
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
1
0

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
2
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
1
1
5
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
2
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
2
0
0
0
0
0
0
0
0
0
0
0
0
10
0
2
0
0
0
0
0
0
0
0
0
0
0
0
1
3
4
7
0
0
0
1
0
0
0
0
0
0
0
0
0
1
1
2
0
3
0
0
0
0
0
0
0
0
0
2
0
3
0
24
1
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
14
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
2
3
5
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
1
7
5
3
0
1
0
6
0
0
0

2
0
2
0
0
2
0
0
0
0
0
0
0
0
0
0
1
0
0
7
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
1
0
0
1
2
1
0
0
0
0
0
10
4
0
2
0
0
2
0
0
0
0
0
4
0
6
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
1
0
0
0
1
0
0
1
0
0
0
0
0
0
1
0
2
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
8
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
2
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
3
1
0
0
0
0
0
0
0
0
0
0
0
1
3
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
8
6
0
0
0
0
0
0
0
0
0
0
4
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
1
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
2
1
0
1
0
0
1
0
0
0
0
0
1
1
4
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
4
0
0
0
0
0
0
0
0
0
0
3
5
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0

0
0
0
0
0
0
0
0
0
1
4
5
7
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
12
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
4
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
10
7
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
9
2
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
8
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
0
0
1
0
0
0
0
0
0
0
0
0
0
7
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
9
1
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
3
0
1
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
13
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
3
0
4
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
2
9
4
1
2
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
4
3
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
5
0
0
0
0
0
0
0
0
0
0
0
2
0
2
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0

0
0
0
0
0
1
0
0
0
0
4
0
2
0
4
0
0
1
0
0
0
0
0
0
1
0
0
0
0
4
0
0
0
0
0
0
0
0
0
0
0
0
4
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
2
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
5
0
0
0
0
0
0
0
0
1
0
2
0
0
0
1
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
3
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
6
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
4
3
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
0
0
0
1
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
2
0
0
0
0
1
1
1
2
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
2
0
0
2
0
0
0
0
0
0
3
1
0
0
0
0
0
0
0
0
0
0
2
0
1
0
0
2
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
2
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
3
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [178]:
img_1.drop(['fire_start', 'fire_end'], axis = 1).to_csv('img_1_with_labels.csv', index = False)

In [168]:
img_2['fire_counts'] = img_2.apply(lambda r: custom_merge(r, viirs), axis = 1)

0
0
0
0
0
0
0
0
0
0
1
0
0
0
2
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
1
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
1
0
0
0
0
0
0
0
0
1
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
2
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
2
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
2
0
0
0
0
0
0
0
0
3
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
1
0
0
1
2
0
1
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
4
0
11
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
1
1
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
1
3
1
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
1
2
0
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
0
1
1
0
0
0
0
0
0
0
0
0
0
0
2
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
5
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
0
0
0
0
0
0
0
0
0
0
1
5
5
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
1
2
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
1
1
0
1
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
1
0
0
0
1
2
0
0
0
0
0
0
0
0
0
0
0
1
0
1
1
0
2
2
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
1
1
1
2
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
2
2
0
1
0
0
0
0
0
0
0
0
3
0
4
1
1
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
2
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
3
0
1
2
3
1
0
0
0
0
0
0
0


0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
2
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
2
0
0
0
0
0
0
0
0
1
0
0
1
0
2
0
0
0
0
0
0
0
0
0
1
5
2
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
1
1
0
0
0
0
0
1
3
0
0
0
0
0


0
1
0
0
1
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
4
6
1
0
0
0
0
0
0
0
0
0
2
0
3
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
4
0
0
1
1
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
1
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
6
11
0
0
0
0
0
0
0
0
0
0
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
3
1
6
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
1
1
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
3
0
6
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
2
0
0
1
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
17
0
0
0
0
0
0
0
0
0
2
4
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
4
0
0
0
2
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
3
6
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
7
2
0
0
0
0
0
0
0
0
3
0
0
3
0
1
0
0


0
0
9
4
5
0
0
0
0
0
0
0
1
0
0
0
0
0
3
2
5
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
0
0
2
5
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
14
0
0
0
0
0
0
0
0
0
0
0
0
0
4
0
0
0
0
0
0
0
0
0
0
0
0
0
2
6
2
1
0
0
0
0
0
1
0
0
0
5
0
0
0
0
0
0
0
0
0
0
1
3
0
0
5
3
0
0
0
0
0
0
0
0
0
0
0
1
15
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
6
1
1
1
0
0
0
0
0
0
0
0
0
0
0
1
0
1
7
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
2
7
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
1
0
8
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
4
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
2
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
7
0
0
0
0
1
0
0
0
0
0
0
0
0
10
0
0
0
0
0
0
0
0
0
0
0
0
3
4
7
6
2
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
5
7
0
0
0
0
0
0
0
0
0
0
0
0
14
4
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
3
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
4
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
8
4
0
0
0
0
0
0
0
0
0
0
0
0
0


...

0
0
0
0
3
5
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
2
11
1
0
0
0
0
0
0
0
0
0
0
0
7
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
8
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
0
0
0
0
0
0
0
0
0
0
0
1
4
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
7
2
0
0
0
0
0
0
0
0
0
0
0
4
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
6
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
4
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
5
0
0
0
0
0
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
7
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
5
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
5
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
4
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
7
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0

6
19
0
0
0
0
0
0
0
0
0
0
6
8
1
4
0
0
0
0
0
0
0
0
0
0
2
8
2
0
0
0
0
0
0
0
0
0
0
1
0
0
3
51
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
25
0
0
0
0
0
37
21
0
0
0
0
0
15
0
11
0
12
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
18
0
0
0
0
0
0
0
0
0
0
2
1
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
5
19
2
0
0
0
0
0
0
0
0
0
0
0
0
0
3
15
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
15
0
0
0
0
0
13
2
0
1
0
0
0
6
0
6
0
0
0
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
5
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
8
3
1
0
0
0
0
0
0
0
0
0
0
0
0
1
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
4
0
0
0
0
0
2
6
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
3
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
4
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
7
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
5

0
0
0
0
0
0
0
0
0
0
0
0
0
0
4
0
0
0
0
0
1
8
0
0
0
0
0
1
0
0
0
0
0
0
0
0
4
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
14
0
0
0
0
0
0
0
0
0
0
4
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
7
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
5
5
0
0
0
0
0
0
0
3
0
0
0
0
0
0
6
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
5
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
11
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
4
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
3
0
0
0
0
0
0
0
2
0
6
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
6
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
3
0
0
0
0
0
0
0
0
0
0
0
4
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
1
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
7
0
0
0
0
0
0
0
0
0
0
0
0
0
1
2
0
0
0
0
0
0
0
0
0
0
0
1
7
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
3
0
0
0
0
0
0
0
0


0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
15
7
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
4
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
4
0
0
0
0
0
0
0
1
0
5
0
0
0
0
0
0
0
0
6
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
3
0
0
0
0
0
0
0
0
0
1
0
0
1
0
0
0
0
0
0
0
0
0
0
6
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
3
0
0
0
0
0
0
0
0
0
0
0
0
9
1
0
0
0
0
0
0
0
0
0
0
0
6
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
4
0
2
1
0
0
0
0
0
0
0
0
0
0
7
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
7
0
0
0
0
0
0
0
0
0
0
0
0
3
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
4
0
0
1
0
0
0
0
0
0
0
0
0
0
3
1
0
0
0
0
0
0
0
0
0
0
0
0
0
4
5
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
4
1
6
1
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
5
1
1
0
0
0
0
0
0
0
0
0
0
2
9
0
0
0
0
0
0
0
0
0
0
0
0
1
2
2
1
0
0
0
0
0
0
0
0
0
0
0
2
3
9
0
0
0
0
0
0
0
0
1
0
1
0
1
0
0
0
0
0
0
0
0

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
9
4
2
2
0
2
0
0
0
0
0
0
0
1
5
0
0
0
0
0
0
0
0
0
0
0
0
0
2
3
3
3
0
0
0
0
0
0
0
0
0
2
0
0
4
0
0
0
0
0
0
0
0
0
0
1
4
5
1
0
0
0
0
0
0
0
0
0
0
0
7
2
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
24
0
0
0
1
0
0
0
0
0
0
0
0
0
34
0
0
0
0
0
0
0
0
0
0
0
0
0
3
26
6
3
0
3
0
0
0
0
0
0
0
0
1
0
7
0
0
0
0
0
0
0
0
0
0
4
0
1
0
1
0
0
0
0
0
0
0
0
1
1
3
1
9
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
17
0
1
1
0
0
0
0
0
0
0
0
0
0
23
0
0
0
0
0
0
0
0
0
0
0
0
0
9
19
1
2
3
0
0
0
0
0
0
0
0
0
2
1
5
0
0
0
0
0
0
0
0
0
0
7
5
10
7
1
0
0
0
0
0
0
0
0
0
0
16
11
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
15
0
2
1
1
2
0
0
0
0
0
0
0
0
25
0
1
0
0
0
0
0
0
0
0
0
0
0
1
8
2
2
0
4
0
0
0
0
0
0
0
3
2
3
15
0
0
0
0
0
0
0
0
0
0
2
7
3
0
0
0
0
0
0
0
0
0
0
0
0
4
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
7
0
0
0
1
0
0
0
0
0
0
0
1
0
16
1
1
0
0
0
0
0
0
0
0
0
0
1
2
13
0
0
0
0
0
0
0
0
0
0
0
4
3
0
4
1
0
0
0
0
0
0
0
2
0
1
2
3
3
0
0
0
0
0
0
0
0
0
1
1
2
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [179]:
img_2.drop(['fire_start', 'fire_end'], axis = 1).to_csv('img_2_with_labels.csv', index = False)

In [172]:
img_3['fire_counts'] = img_3.apply(lambda r: custom_merge(r, viirs), axis = 1)

0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
0
1
0
0
0
0
0
0
0
0
3
3
3
2
0
0
0
0
0
0
1
0
0
0
0
0
1
4
0
4
2
7
2
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
1
0
0
0
0
0
0
0
0
1
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
4
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
12
0
0
0
0
0
0
0
0
0
1
1
10
0
1
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
5
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
8
5
1
0
1
0
0
0
0
0
0
0
0
0
0
0
32
0
0
0
0
0
0
0
0
0
1
1
6
0
0
0
0
0
0
0
0
0
0
3
1
4
0
0
0
0
0
0
0
0
0
0
0
1
7
4
0
0
0
0
0
0
0
0
0
0
0
0
0
3
16
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
2
7
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
1
0
5
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
5
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
4
0
0
0
0
0
0
0
0
0
0
0
0
0
12
0
0
0
0
0
0
0
0
0
0
0
21
0
0
0
0
0
0
0
0
0
0
1
0
0
4
0
0
0
0
0
0
0
0
0
0
5
0
18
0
0
0
0
0
0
0
0
0
0
0
0
0
0
35
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
1
1
0
0
0
0
0
0
0
0
0
0
0
0
58
0
0
0
0
0
0
0
0
0
0
0
3
0
4
0
0
0
0
0
0
0
0
0
0
1
6
0
0
0
0
0
0
0
0
0
0
2
1
4
0
0
0
0
0
0
0
0
0
0
0
0
0

0
6
2
1
0
0
0
0
0
0
0
0
0
0
0
0
0
49
0
0
0
0
0
0
0
0
0
0
1
7
0
1
0
0
0
0
0
0
0
0
1
0
0
8
0
0
0
0
0
0
0
0
0
0
8
1
24
1
0
0
0
0
0
0
0
0
0
0
0
0
8
37
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
9
0
0
2
0
0
0
0
0
0
0
0
0
0
0
25
0
0
0
0
0
0
0
0
0
0
2
5
0
0
0
0
0
0
0
0
0
0
3
0
1
12
0
0
0
0
0
0
0
0
0
0
2
3
8
0
0
0
0
0
0
0
0
0
0
0
0
0
1
8
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
7
0
0
0
0
0
0
0
0
0
0
0
0
0
0
24
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
2
1
0
13
1
0
0
0
0
0
0
0
0
0
4
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
18
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
11
1
0
0
0
0
0
0
0
0
0
0
1
0
0
18
0
0
0
0
0
0
0
0
1
0
0
1
0
0
0
0
0
0
0
2
0
0
2
0
0
18
0
0
0
0
1
0
0
0
0
0
1
0
2
3
0
0
0
0
0
0
0
0
0
3
1
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
4
0
0
0
0
0
0
0
0
0
0
0
1
0
0
51
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
4
0
0
0
0
0
0
0
0
0
0
0
0
4
0
0
0
0
0
0
0
0
0
0
0
3
0
0
14
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
3
1
0
0
0
0
0
0
0
0
0
0

0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
9
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
29
0
0
0
0
0
0
0
0
0
0
0
2
0
2
0
0
0
0
0
0
0
0
0
0
0
8
0
0
0
0
0
0
0
0
0
0
2
0
7
0
0
0
0
0
0
0
0
0
0
0
0
0
0
29
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
6
9
0
0
0
0
0
0
0
0
0
0
0
0
0
0
52
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
18
0
0
0
0
0
0
0
0
0
0
1
1
7
1
0
0
0
0
0
0
0
0
0
0
0
0
0
19
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
15
0
0
0
0
0
0
0
0
0
0
1
5
0
0
0
0
0
0
0
0
0
0
0
0
0
6
0
0
0
0
0
0
0
0
0
0
1
4
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
5
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
4
5
0
1
0
0
0
0
0
0
0
0
0
0
0
0
20
0
0
0
0
0
0
0
0
1
0
0
4
1
0
0
0
0
0
0
0
0
0
0
0
1
17
0
0
0
0
0
0
0
0
0
0
2
2
7
1
0
0
0
0
0
0
0
0
0
0
1
0
0
7
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
5
0
0
0
0
0
0
0
0
0
0
0
0
0
0
10
0
0
0
0
0
0
0
0
0
0
0
3
2
0
0
0
0
0
0
0
0
0
0
0
5
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
28
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
7
2
3
1
0
0
0
0
0
0
0
0
0
0
0
3
2
0
2
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
8
4
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
8
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
5
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
4
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
5
15
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
2
29
4
0
0
0
0
0
0
0
0
0
0
0
0
3
0
32
1
0
0
0
0
0
0
0
0
0
0
0
0
0
4
12
0
0
0
0
0
1
0
0
0
0
0
0
0
0
3
24
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
6
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
4
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
14
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0

...

0
0
0
0
0
0
0
0
0
0
2
1
3
2
0
0
0
0
0
0
0
0
0
0
0
3
6
13
0
0
0
0
0
0
0
0
0
0
0
0
0
0
11
9
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
31
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
9
0
1
0
0
0
0
0
0
0
0
0
0
0
0
2
4
0
2
0
0
0
0
0
0
0
0
0
0
0
0
4
12
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
13
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
24
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
6
1
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
1
2
0
0
0
0
0
0
0
0
0
0
0
0
0
5
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
4
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
8
62
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
2
2
0
2
0
0
0
0
0
0
0
0
0
0
1
1
6
0
6
3
0
2
0
0
0
0
0
0
0
0
0
1
7
5
1
0
0
0
0
0
0
0
0
0
0
0
0
0
36
4
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
41
0
0
0
0
0
0
0
0
0
0
0
0
0
0
69
0
0
0
0
0
0
0
0
0
0
0
3
0
1
1
0
0
0
0
0
0
0
6
5
1
9


0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
16
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
2
0
3
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
9
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
4
0
1
0
0
0
0
0
0
0
0
0
0
1
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
5
1
0
0
1
0
0
0
0
0
0
0
0
0
0
0
4
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
6
6
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
13
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
8
1
0
0
0
0
0
0
0
0
0
0
0
0
2
4
1
4
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
1
0
0
0
0
0
0
0
0
0
0
0
0
4
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
7
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
20
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
17
7
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
2
2
5
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
6
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
11
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
21


0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
7
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
4
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
6
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
5
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
4
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
7
1
2
0
0
0
0
0
0
0
0
0
0
1
0
0
0
5
4
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
5
6
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
37
0
0
0

0
0
0
0
0
0
0
0
0
0
0
0
0
0
20
1
0
0
0
0
0
0
0
0
0
0
0
0
5
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
2
11
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
10
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
13
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
16
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
37
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
20
7
2
0
0
0
0
0
0
0
0
0
0
0
0
2
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
1
13
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
6
53
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
44
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
13
5
1
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
19
6
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
13
0
3
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
39
1
0
0
0
0
0


0
0
0
0
0
0
3
8
4
1
0
0
0
0
0
0
0
0
0
0
0
2
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
19
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
16
2
0
0
0
0
0
0
0
0
0
0
0
0
4
4
1
0
0
0
0
0
0
0
0
0
0
0
14
1
1
0
0
0
0
0
0
0
0
0
0
0
6
3
2
0
0
0
0
0
0
0
0
0
0
0
0
4
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
20
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
33
10
0
0
0
0
0
0
0
0
0
0
0
0
6
16
8
2
0
0
0
0
0
0
0
0
1
0
5
0
1
0
0
0
0
0
0
0
0
0
0
0
10
8
6
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
8
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
15
3
0
0
0
0
0
0
0
0
0
0
0
0
3
4
2
0
0
0
0
0
0
0
0
0
0
0
3
1
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
26
4
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
29
3
0
0
0
0
0
0
0
0
0
0
0
0
12
3
4
0
0
0
0
0
0
0
0
0
1
0
7
2
0
0
0
0
0
0
0
0
0
0
0
0
11
4
1
0
0
0
0
0
0
0
0
0
0
0
1
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
5
0
0
0
0
0
0
0
0
0
0
0
0
0
2
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0

In [180]:
img_3.drop(['fire_start', 'fire_end'], axis = 1).to_csv('img_3_with_labels.csv', index = False)

In [176]:
img_4['fire_counts'] = img_4.apply(lambda r: custom_merge(r, viirs), axis = 1)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
4
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


0
0
0
0
0
0
0
0
0
0
0
6
5
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
6
8
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
11
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
3
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
9
22
1
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
6
13
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
42
0
0
0
0
0
0
0
0
0
0
0
0
0
2
1
0
0
0
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
8
17
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
15
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
9
0
0
0
0
0
0
0
0
0
0
0
0
1
11
9
1
0
0
0
0
0
0
0
0
0
1
0
3
1
0
0
0
0
0
0
0
0
0
0
0
0
1
2
9
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
5
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
10
2
0
0
0
0
0
0
0
0
0
0
0
0
5
13
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
6
5
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
4
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
13
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
17
0
0
0
0
0
0
0
0
0
0
0
0
0
1
10
2
0
0
0
0
0
0
0
0
0
0
0
8
0


0
0
0
0
0
0
0
0
0
4
1
0
0
0
0
0
0
0
0
0
1
0
0
4
7
0
0
0
0
0
0
0
3
0
1
0
1
0
0
0
0
0
0
0
0
0
0
0
1
3
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
8
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
17
0
0
0
0
0
0
0
0
0
0
0
0
0
4
4
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
2
1
0
0
0
0
0
0
0
0
0
0
0
0
0
23
5
1
0
0
0
0
0
0
0
0
0
0
0
0
4
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
2
5
2
2
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
22
9
0
0
0
0
0
0
0
0
0
0
0
0
5
14
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
4
0
0
0
0
0
0
0
0
1
0
0
0
0
0
22
16
2
0
0
0
0
0
0
0
0
0
0
0
0
0
5
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
22
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
21
5
0
0
0
0
0
0
0
1
0
0
0
2
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
10
17
1
0
0
0
0
0
0
0
0
0
0
0
0
0
6
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
16
5
0
0
0
0
0
0
0
0
0
0
0
0
14
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
10
3
0
0
0
0
0
0
0
0
0
0
0
0
0

0
0
0
0
0
0
0
0
0
0
0
0
5
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
17
0
0
0
0
0
0
0
0
0
0
0
0
0
3
1
0
0
0
0
0
0
0
0
0
0
1
0
1
1
3
0
0
0
0
0
0
0
0
0
0
0
6
2
0
0
0
0
0
0
0
0
0
0
0
0
1
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
14
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
40
2
0
0
0
0
0
0
0
0
0
0
0
0
19
8
2
0
0
0
0
0
0
0
0
0
0
0
0
11
0
0
0
0
0
0
0
0
0
0
0
1
6
1
9
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
5
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
12
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
5
15
11
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
17
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
8
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
5
5
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
6
20
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
7
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
17
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
10
3


0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
4
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
6
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
4
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
5
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
2
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
5
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
14
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0

...

0
0
2
45
2
0
0
0
0
0
0
0
0
0
0
0
0
9
6
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
7
0
0
0
0
0
0
0
0
0
0
0
0
0
2
3
10
0
0
0
0
0
0
0
0
0
0
0
0
0
2
4
7
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
6
7
1
3
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
44
5
0
0
0
0
0
0
0
0
0
0
0
0
16
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
5
10
0
0
0
0
0
0
0
0
0
0
0
0
0
0
5
3
1
0
0
0
0
0
0
0
0
0
0
0
0
4
2
2
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
5
5
3
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
47
4
0
0
0
0
0
0
0
0
0
0
0
0
5
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
7
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
5
1
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
5
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
28
17
0
0
0
0
0
0
0
0
0
0
0
0
2
2
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
5
0
0
0
0
0
0
0
0
0
0
0
0
0
0
4
14
2
0
0
0
0
0
0
0
0
0
0
0
2
0
0
3
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
7
0
6
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
21
17
2
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
6
1
0
0
0
0
0

0
0
0
0
0
0
0
0
0
9
11
0
0
0
0
0
0
0
0
0
0
0
0
0
2
4
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
23
11
1
0
0
0
0
0
0
0
0
0
0
0
0
0
14
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
24
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
1
15
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
6
17
2
0
0
0
0
0
0
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
16
6
0
0
0
0
0
0
0
0
0
0
0
0
0
0
41
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
41
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
8
0
4
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
4
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
7
0
0
0
0
0
0
0
0
0
0
0
0
0
0
9
17
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
10
0
3
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
2
4
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
7
2
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
7
3
2
1
0
0
0
0
0
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
2
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
1
6
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
0
11
4
5
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
3
2
2
0
0
0
0
0
0
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
1
2
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
5
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
10
7
1
0
0
0
0
0
0
0
0
0
0
0
3
0
1
1
0
0
0
0
1
0
0
0
0
0
0
0
0
1
1
0
0
7
0
0
0
0
0
0
0
0
0
0
0
0
2
1
0
0
0
0
0
0
0
0
0
0
0
0
0
3
1
1
0


0
0
0
0
0
0
0
0
0
0
0
0
0
8
1
11
3
1
0
1
0
0
0
0
0
0
0
0
1
0
0
1
5
7
1
10
0
0
0
0
0
0
0
0
0
0
2
0
3
4
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
4
11
1
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
0
0
0
1
0
0
0
0
0
0
1
1
2
7
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
3
9
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
3
2
5
0
0
0
0
0
1
0
0
0
0
0
0
1
3
0
0
0
0
0
0
0
0
0
0
0
0
0
5
0
1
1
10
0
0
0
0
0
0
0
0
0
0
0
0
0
0
5
0
0
0
0
0
0
0
0
0
0
0
1
0
0
1
5
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
4
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
4
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
4
1
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
7
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
7
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
3
2
0
0
0
0
0
0
0
0
0
0


0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
8
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
5
1
0
0
0
0
0
0
0
0
0
0
0
0
0
5
1
0
0
0
0
0
0
0
0
0
0
0
0
5
0
0
0
0
0
0
0
0
0
0
0
4
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
8
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
8
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
3
1
24
6
6
1
0
0
0
0
0
0
0
0
0
0
0
0
2
11
0
0
0
0
0
0
0
0
0
0
0
1
4
9
32
0
0
0
0
0
0
0
0
0
0
1
0
2
4
1
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
12
15
4
6
0
0
0
0
0
0
0
0
0
0
0
0
3
1
0
0
0
0
0
0
0
0
0
0
0
2
3
12
11
0
0
0
0
0
0
0
0
0
0
0
1
3
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0

In [181]:
img_4.drop(['fire_start', 'fire_end'], axis = 1).to_csv('img_4_with_labels.csv', index = False)

## Testing

In [38]:
viirs_2019 = pd.read_csv('data/viirs_2019.csv')
viirs_2019.head()

,latitude,longitude,bright_ti4,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_ti5,frp,daynight,type
0,-24.984251,46.381214,340.22,0.48,0.40,2019-01-01,1050,N,VIIRS,n,1,306.14,6.07,D,0
1,-24.980591,46.380432,342.77,0.48,0.40,2019-01-01,1050,N,VIIRS,n,1,307.38,6.07,D,0
2,-25.244823,45.616020,337.53,0.44,0.38,2019-01-01,1050,N,VIIRS,n,1,307.24,3.37,D,0
3,-21.658609,47.599316,330.06,0.41,0.45,2019-01-01,1051,N,VIIRS,n,1,294.55,2.34,D,0
4,-22.198599,47.574085,331.97,0.40,0.44,2019-01-01,1051,N,VIIRS,n,1,290.98,2.87,D,0


In [89]:
vrs = viirs_pipe('data/viirs_2019.csv', 'data/grid.csv')

In [94]:
vrs['acq_date'] = pd.to_datetime(vrs['acq_date'])

In [135]:
img_1 =img_pipe("img/1_img.csv", n_weeks = 5, n_day_range = 7)

In [136]:
test = img_1.tail(200)
test

,rect_id,date,band,count,bin_0,bin_1,bin_2,bin_3,bin_4,bin_5,...,bin_24,bin_25,bin_26,bin_27,bin_28,bin_29,bin_30,bin_31,fire_start,fire_end
749101,513,2018-12-17,B4,5397.309804,0.0,0.000000,0.000000,0.000000,0.000000,18.000000,...,155.396078,198.964706,161.654902,224.690196,177.082353,227.000000,159.341176,210.396078,2019-01-21,2019-01-28
749102,513,2018-12-17,B5,5911.980392,0.0,8.000000,35.745098,117.745098,197.192157,243.298039,...,166.164706,123.874510,77.258824,47.000000,28.000000,7.000000,1.137255,2.121569,2019-01-21,2019-01-28
749103,513,2018-12-17,B6,5914.980392,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2019-01-21,2019-01-28
749104,513,2018-12-17,B7,5914.980392,15.0,221.211765,502.498039,633.917647,853.341176,597.764706,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2019-01-21,2019-01-28
749105,513,2019-01-02,B1,25126.392157,0.0,40.364706,1395.266667,6266.650980,9209.301961,4367.223529,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2019-02-06,2019-02-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749296,513,2020-11-20,B3,54774.035294,0.0,0.000000,125.815686,2915.078431,6478.145098,10770.639216,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2020-12-25,2021-01-01
749297,513,2020-11-20,B4,54774.035294,0.0,0.000000,2.000000,199.000000,488.000000,426.631373,...,2906.713725,1742.721569,930.403922,449.905882,180.521569,44.607843,8.000000,0.000000,2020-12-25,2021-01-01
749298,513,2020-11-20,B5,54772.035294,15.0,621.372549,487.992157,182.768627,169.494118,107.996078,...,697.070588,247.784314,89.125490,28.000000,9.000000,4.000000,0.000000,1.000000,2020-12-25,2021-01-01
749299,513,2020-11-20,B6,54774.035294,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,24951.294118,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2020-12-25,2021-01-01


In [113]:
# imgsmall = img.head(50)
# n_weeks = 5
# n_range = 1
# imgsmall['fire_start'] = pd.to_datetime(imgsmall['date']) + timedelta(n_weeks)
# imgsmall['fire_end'] = pd.to_datetime(imgsmall['date']) + timedelta(n_weeks+n_range)
# imgsmall

In [111]:
img_1['date'] = pd.to_datetime(img_1['date'])   

In [114]:
img_1['fire_start'] = img_1['date'] + timedelta(n_weeks)
img_1['fire_end'] = img_1['date'] + timedelta(n_weeks+n_range)

In [138]:
test.apply(lambda r: custom_merge(r, vrs), axis = 1)

0

In [ ]:
imgsmall.merge(vrs, how = 'left', left_on = 'region')

In [ ]:
viirs_2019

In [39]:
polygon = pd.read_csv('data/mdg_polygon.csv')

def in_polygon(x, y):
    """Whether a longitude-latitude (x, y) pair is in the polygon."""

    j = polygon.shape[0]-1
    odd = False
    for i in range(polygon.shape[0]):
        xsi, xsj, ysi, ysj = xs[i], xs[j], ys[i], ys[j]
        if (ysi < y <= ysj) or (ysj < y <= ysi):
            z = (y - ysi) / (ysj - ysi) * (xsj - xsi)
            if xsi + z < x: 
                odd = not odd
        j = i
    return odd


xs, ys = polygon['lon'], polygon['lat']

def fires_in_polygon(row):
    fire = in_polygon(row['longitude'], row['latitude'])
    return fire



In [40]:
clean_viirs_2019 = viirs_2019[viirs_2019.apply(fires_in_polygon, axis=1)]

In [41]:
clean_viirs_2019

,latitude,longitude,bright_ti4,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_ti5,frp,daynight,type
262,-14.884380,49.972679,345.68,0.38,0.36,2019-01-03,1015,N,VIIRS,n,1,295.10,6.03,D,0
271,-15.070460,50.014191,334.82,0.38,0.36,2019-01-03,1015,N,VIIRS,n,1,295.60,8.48,D,0
274,-14.876871,50.155300,332.15,0.39,0.36,2019-01-03,1015,N,VIIRS,n,1,295.64,2.67,D,0
278,-14.946500,50.251881,332.86,0.39,0.36,2019-01-03,1015,N,VIIRS,n,1,296.16,2.79,D,0
343,-15.793204,50.274464,333.35,0.52,0.42,2019-01-04,956,N,VIIRS,n,1,296.16,9.84,D,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323861,-15.017243,50.205406,334.14,0.48,0.40,2019-12-31,1027,N,VIIRS,n,1,291.94,4.53,D,0
323869,-15.046171,50.101151,344.99,0.47,0.40,2019-12-31,1027,N,VIIRS,n,1,293.51,16.97,D,0
323870,-15.050572,50.097336,331.21,0.47,0.40,2019-12-31,1027,N,VIIRS,n,1,293.22,2.89,D,0
323871,-15.049833,50.101738,341.54,0.47,0.40,2019-12-31,1027,N,VIIRS,n,1,294.38,7.54,D,0


## Marking the Squares

In [44]:
#### helper function for determining whether the coordinate is in the square
def in_square(r_lat, r_lon, sq):
    return ((r_lat >=  sq[0]) & (r_lat <= sq[1]) & (r_lon >= sq[2]) & (r_lon <= sq[3]))


In [70]:
#### function for determining whether it is in the square
def mark_square(r, dict_label_square):
    r_lat = r['latitude']
    r_lon = r['longitude']
    for label, square in dict_label_square.items():
        if in_square(r_lat, r_lon, square):
            return label
    
    return "not found"


In [42]:
#### testing
d_lab_sq = {"A": [-14.89, 49.972679, -14.33, 50],
           "B": [-15.070460, 50.014191, -14.24, 51]}

In [46]:
small_clean_viirs_2019 = clean_viirs_2019.head(10)
small_clean_viirs_2019

,latitude,longitude,bright_ti4,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_ti5,frp,daynight,type
262,-14.884380,49.972679,345.68,0.38,0.36,2019-01-03,1015,N,VIIRS,n,1,295.10,6.03,D,0
271,-15.070460,50.014191,334.82,0.38,0.36,2019-01-03,1015,N,VIIRS,n,1,295.60,8.48,D,0
274,-14.876871,50.155300,332.15,0.39,0.36,2019-01-03,1015,N,VIIRS,n,1,295.64,2.67,D,0
278,-14.946500,50.251881,332.86,0.39,0.36,2019-01-03,1015,N,VIIRS,n,1,296.16,2.79,D,0
343,-15.793204,50.274464,333.35,0.52,0.42,2019-01-04,956,N,VIIRS,n,1,296.16,9.84,D,0
344,-15.794609,50.271717,334.54,0.52,0.42,2019-01-04,956,N,VIIRS,n,1,295.72,6.44,D,0
350,-14.875433,50.154743,333.56,0.52,0.41,2019-01-04,956,N,VIIRS,n,1,295.07,4.66,D,0
355,-15.040085,50.105476,333.87,0.52,0.42,2019-01-04,956,N,VIIRS,n,1,293.63,4.72,D,0
371,-15.365170,50.450977,333.86,0.34,0.56,2019-01-05,937,N,VIIRS,n,1,291.91,2.12,D,0
372,-15.350317,50.442894,335.04,0.34,0.56,2019-01-05,937,N,VIIRS,n,1,291.56,2.85,D,0


In [44]:
small_clean_viirs_2019.apply(lambda r: mark_square(r, d_lab_sq), axis=1)

262            A
271            B
274            B
278            B
343    not found
344    not found
350            B
355            B
371    not found
372    not found
dtype: object

### Reading the Grid

In [8]:
grid = pd.read_csv('data/grid.csv', header = None)
grid

,0,1,2,3
0,"46.1374729923786, -24.7935018416987","46.2093382151081, -24.7935018416987","46.2093382151081, -24.7216366189692","46.1374729923786, -24.7216366189692"
1,"46.1374729923786, -24.8653670644283","46.2093382151081, -24.8653670644283","46.2093382151081, -24.7935018416987","46.1374729923786, -24.7935018416987"
2,"46.1374729923786, -24.9372322871579","46.2093382151081, -24.9372322871579","46.2093382151081, -24.8653670644283","46.1374729923786, -24.8653670644283"
3,"46.2093382151081, -24.7935018416987","46.2812034378377, -24.7935018416987","46.2812034378377, -24.7216366189692","46.2093382151081, -24.7216366189692"
4,"46.2093382151081, -24.8653670644283","46.2812034378377, -24.8653670644283","46.2812034378377, -24.7935018416987","46.2093382151081, -24.7935018416987"
...,...,...,...,...
2049,"50.3056559106932, -15.5947533323148","50.3775211334227, -15.5947533323148","50.3775211334227, -15.5228881095853","50.3056559106932, -15.5228881095853"
2050,"50.3775211334227, -15.1635619959375","50.4493863561523, -15.1635619959375","50.4493863561523, -15.0916967732079","50.3775211334227, -15.0916967732079"
2051,"50.3775211334227, -15.235427218667","50.4493863561523, -15.235427218667","50.4493863561523, -15.1635619959375","50.3775211334227, -15.1635619959375"
2052,"50.3775211334227, -15.3072924413966","50.4493863561523, -15.3072924413966","50.4493863561523, -15.235427218667","50.3775211334227, -15.235427218667"


In [10]:
def parse_grid_file(grid_file: str):
    df = pd.read_csv(grid_file)
    vals = df.values
    str_to_arr = lambda s: np.array([float(x) for x in s.split(',')])
    vals = np.array([[str_to_arr(x2) for x2 in x1] for x1 in vals])
    return vals

In [12]:
grid_arr = parse_grid_file('data/grid.csv')

In [57]:
np.sort(list(set(reshaped_grid[0])))

array([-24.86536706, -24.79350184,  46.13747299,  46.20933822])

In [58]:
reshaped_grid = grid_arr.reshape(-1,8)
grid_dict = {}
for i in range(len(reshaped_grid)):
    lat_lon = np.sort(list(set(reshaped_grid[i])))
    grid_dict[i] = lat_lon

In [60]:
small_clean_viirs_2019['region'] = small_clean_viirs_2019.apply(lambda r: mark_square(r, grid_dict), axis=1)

/Users/mananahakobyan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [69]:
n_days = 14
small_clean_viirs_2019['match_date'] = pd.to_datetime(small_clean_viirs_2019['acq_time']) - timedelta(n_days)

/Users/mananahakobyan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
